# Two-Stage Filter Experiment

This notebook tests the hypothesis that **QR codes appear as "semi-bright" regions (0.4 - 0.6)** in the Edge Density map.

We implement a two-stage pipeline:
1.  **Stage 1: Edge Density**: Computes the density of edges in the image.
2.  **Stage 2: Intensity Band-Pass**: Highlights pixels in the Stage 1 output that fall within a specific range (e.g., 0.4 to 0.6).

This cascading approach aims to isolate QR codes by filtering out very low texture (background) AND very high texture (text/complex noise) if they differ in density intensity.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path

%matplotlib inline

## 1. Filter Implementations

In [2]:
def normalize01(x: np.ndarray) -> np.ndarray:
    if x is None: return None
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx - mn < 1e-6:
        return np.zeros_like(x, dtype=np.float32)
    return (x - mn) / (mx - mn)

# --- Stage 1: Edge Density ---
def filter_edge_density(gray, thresh1=50, thresh2=150, radius=15):
    edges = cv2.Canny(gray, thresh1, thresh2)
    edges_f = (edges > 0).astype(np.float32)
    # Box filter for density
    d_k = radius * 2 + 1
    density = cv2.blur(edges_f, (d_k, d_k))
    return normalize01(density)

# --- Stage 2: Band-Pass Filter ---
def filter_band_pass(score_map, min_val=0.4, max_val=0.6):
    """
    Returns a binary-like map (0.0 to 1.0) where the input score_map
    is between min_val and max_val.
    """
    # Create a mask
    mask = (score_map >= min_val) & (score_map <= max_val)
    return mask.astype(np.float32)

## 2. Interactive Experiment

In [ ]:
# Load Images
data_dir = Path("../data/raw")
files = sorted(list(data_dir.glob("*.png")) + list(data_dir.glob("*.jpg")) + list(data_dir.glob("*.jpeg")))
options = [(p.name, str(p)) for p in files]
if not options: options = [("No images", None)]

def show_two_stage(
    image_path,
    # Stage 1: Edge Density
    e_th1, e_th2, e_rad,
    # Stage 2: Band Pass
    band_min, band_max,
    # Extraction
    patch_size, top_k
):
    if not image_path: return
    
    img = cv2.imread(image_path)
    if img is None: return
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # --- Stage 1: Edge Density ---
    s1_map = filter_edge_density(gray, thresh1=e_th1, thresh2=e_th2, radius=e_rad)
    
    # --- Stage 2: Band Pass ---
    # Input is the Output of Stage 1
    s2_map = filter_band_pass(s1_map, min_val=band_min, max_val=band_max)
    
    # --- Patch Extraction (from Stage 2 map) ---
    # Reuse basic extraction logic for visualization
    h, w = s2_map.shape[:2]
    nms_map = s2_map.copy()
    half = patch_size // 2
    out_img = img_rgb.copy()
    count = 0
    
    # Simple Greedy NMS for visualization speed
    for _ in range(top_k):
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(nms_map)
        if max_val < 0.5: break # Binary map, so we look for 1.0 (or close to it)
        
        cx, cy = max_loc
        x1 = max(0, cx - half)
        y1 = max(0, cy - half)
        x2 = min(w, x1 + patch_size)
        y2 = min(h, y1 + patch_size)
        
        cv2.rectangle(out_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        pad = int(patch_size * 1.0) # Standard suppression
        nx1 = max(0, cx - pad//2); ny1 = max(0, cy - pad//2)
        nx2 = min(w, cx + pad//2); ny2 = min(h, cy + pad//2)
        nms_map[ny1:ny2, nx1:nx2] = 0
        count += 1

    # --- Visualization ---
    fig, axes = plt.subplots(1, 4, figsize=(24, 6))
    
    # 1. Original
    axes[0].imshow(img_rgb)
    axes[0].set_title("Original")
    axes[0].axis('off')
    
    # 2. Stage 1 (Edge Density)
    im1 = axes[1].imshow(s1_map, cmap='jet')
    axes[1].set_title("Stage 1: Edge Density (0.0 - 1.0)")
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
    
    # 3. Stage 2 (Band Pass)
    im2 = axes[2].imshow(s2_map, cmap='gray')
    axes[2].set_title(f"Stage 2: Band Pass ({band_min} - {band_max})")
    axes[2].axis('off')
    plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
    
    # 4. Result Boxes
    axes[3].imshow(out_img)
    axes[3].set_title(f"Result: {count} Boxes")
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()

interact(
    show_two_stage, 
    image_path=widgets.Dropdown(options=options, description="Image:"),
    
    e_th1=widgets.IntSlider(value=50, min=10, max=200, description='Canny Th1'),
    e_th2=widgets.IntSlider(value=150, min=50, max=300, description='Canny Th2'),
    e_rad=widgets.IntSlider(value=15, min=3, max=50, description='Edge Radius'),
    
    band_min=widgets.FloatSlider(value=0.4, min=0.0, max=1.0, step=0.05, description='Band Min'),
    band_max=widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05, description='Band Max'),
    
    patch_size=widgets.IntSlider(value=128, min=32, max=256, step=32, description='Patch Size'),
    top_k=widgets.IntSlider(value=20, min=1, max=100, description='Max Boxes'),
);

interactive(children=(Dropdown(description='Image:', options=(('001.png', '../data/raw/001.png'), ('002.png', …